In [1]:
# --- Requirements --- 

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [2]:
# --- loading data --- 
filepath = "../Datasets/hour.csv"
df = pd.read_csv(filepath)

#  --- defining features --- 
drop_cols = ['temp', 'instant', 'dteday', 'casual', 'registered', 'cnt']
numeric_features = ['atemp', 'hum', 'windspeed'] 
categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']

#  --- deleting unwanted features --- 
X = df.drop(columns=drop_cols)
y = df['cnt']

#  --- safety for keeping only the defined features --- 
X = X[numeric_features + categorical_features]

#  --- splitting data --- 
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# --- preprocessing and model pipeline ---

# --- safety check: removing constant columns --- 
const_cols = X_train.columns[X_train.nunique() <= 1].tolist()
if const_cols:
    print(f"Remove const columns:", const_cols)
    X_train = X_train.drop(columns=const_cols)
    X_test = X_test.drop(columns=const_cols)

num_cols = [c for c in numeric_features if c in X_train.columns]
cat_cols = [c for c in categorical_features if c in X_train.columns]

# --- numerical features: imputation + standard scaling --- 
num_pipeline = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='mean')),
        ('scalar', StandardScaler())
    ]
)

# --- categorical features: imputation + one hot encoding --- 
cat_pipeline = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

# --- Column Transformer: num_pipeline auf num_cols, cat_pipeline auf cat_cols --- 
preproc = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ],
    remainder='drop' # alle anderen Spalten verwerfen
)

# --- acutual pipeline, combining preprocessing and linear Regression model --- 
RF_model = Pipeline(
    [
        ('preproc', preproc),
        ('rf', RandomForestRegressor(
            n_estimators=200,
            random_state=42,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=2))
    ]
)

# --- fitting the pipeline ---
RF_model.fit(X_train, y_train)

print("\033[94m\n\n----- Pipeline fitted -----\033[0m")



----- Pipeline fitted -----


In [4]:
#  --- Evaluation on test set --- 
y_pred = RF_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"R2 on test:     {r2:.3f}")
print(f"RMSE on test:   {rmse:.3f}")
print(f"MAPE on test:   {mape:.3f} ({mape*100:.1f}%)")

R2 on test:     0.842
RMSE on test:   70.761
MAPE on test:   0.555 (55.5%)
